In [2]:
import pandas as pd
import numpy as np
import ast
from tqdm import tqdm
import swifter
import json
tqdm.pandas()

In [7]:
df = pd.read_csv('data/995,000_row_cleaned.csv')

/var/folders/z1/hf1tn63x7tl1qjxy1md9xy4w0000gn/T/ipykernel_47904/1234656882.py:1: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/995,000_row_cleaned.csv')


In [5]:
test_df = pd.read_csv('data/news_sample_6.csv')

In [9]:
# Remove columns 
test_df = test_df[['title', 'content', 'scraped_at', 'inserted_at', 'url', 'keywords']]

In [ ]:
# Create vocab
vocab = {}
def create_vocab(lst):
    lst = ast.literal_eval(lst)
    for word in lst:
        if word in vocab:
            vocab[word] += 1
        else:
            vocab[word] = 1

df['content'].swifter.progress_bar(True).apply(create_vocab)
print(vocab)

In [ ]:
# Shorten the vocab to 10,000 most common words
shortend_vocab = dict(sorted(vocab.items(), key=lambda item: item[1], reverse=True)[:10000])
with open('10000vocab.json', 'w') as f:
    json.dump(shortend_vocab, f)

In [ ]:
# Load vocab
with open('10000vocab.json', 'r') as f:
    vocab = json.load(f)

{'num': 3357809, 'said': 1415604, 'one': 1120226, 'new': 1060598, 'us': 982961, 'would': 961441, 'state': 945863, 'peopl': 921858, 'year': 921313, 'time': 903363, 'like': 841154, 'use': 705869, 'also': 693032, 'make': 624966, 'say': 600075, 'mr': 596570, 'go': 585354, 'get': 582400, 'even': 572394, 'presid': 541115, 'govern': 539747, 'work': 525346, 'american': 521129, 'report': 518587, 'nation': 511747, 'right': 510396, 'first': 502170, 'trump': 499711, 'day': 499467, 'two': 495840, 'news': 489923, 'could': 489224, 'come': 489062, 'mani': 488774, 'obama': 488258, 'world': 460704, 'take': 455102, 'continu': 454920, 'stori': 450569, 'call': 438802, 'read': 438779, 'last': 435706, 'see': 426534, 'want': 426267, 'may': 424096, 'know': 415252, 'includ': 414502, 'way': 411852, 'think': 411722, 'countri': 403293, 'back': 400971, 'need': 393197, 'public': 391768, 'support': 384895, 'polit': 377828, 'well': 377769, 'unit': 366068, 'look': 360407, 'tri': 358077, 'iran': 357510, 'war': 355046, '

In [11]:
def count_freq(lst):
    dic = {key: 0 for key in list(vocab)}
    lst = ast.literal_eval(lst)
    for word in lst:
        if word in dic:
            dic[word] += 1
    return [value for value in dic.values()]
test_df['content'] = test_df['content'].swifter.progress_bar(True).apply(count_freq)

# Save cleaned data
output_path = 'data/news_sample_7.csv'
test_df.to_csv(output_path, index=False)
print(f"Fully cleaned; data saved to {output_path}")

Pandas Apply: 100%|██████████| 250/250 [00:00<00:00, 1607.88it/s]


Fully cleaned; data saved to data/news_sample_7.csv


In [ ]:
label_map = {"fake": 1, "satire":1, "conspiracy": 1, "unreliable": 1, "bias": 1, "rumor": 1, "junksci": 1, "hate": 1,
             "reliable": 0,  "clickbait": 0,   "political": 0}
labels = np.array([label_map.get(row['type']) for _,row in test_df.iterrows()])
features = np.array([count_freq(row['content']) for _,row in test_df.iterrows()])
dataset = np.column_stack((labels, features))

# Print shapes
print("Features shape:", features.shape)  # (num_samples, num_features)
print("Labels shape:", labels.shape)  # (num_samples,)

# Example output
print("First sample:", dataset[-1])

['2017-11-27T01:14:42.983556' '2017-11-27T01:14:08.7454'
 '2017-11-27T01:14:21.395055' ... '2018-02-11 00:36:05.970564'
 '2018-02-11 00:36:39.518922' '2018-02-11 00:44:17.105681']
